In [1]:
import sys
from tensorflow import keras
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD


In [2]:
def load_dataset():
    # loading of the data
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    print(f'Train : X = {trainX.shape}, Y = {trainY}')
    print(f'Test : X = {testX.shape}, Y = {testY}')

    # one hot encoding for categorical data (only output)

    trainY = keras.utils.to_categorical(trainY)
    testY = keras.utils.to_categorical(testY)
    return trainX, trainY, testX, testY

In [3]:
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm  = test.astype('float32')

    # normalize between 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0

    # returning scaled images or normalized images
    return train_norm, test_norm

In [4]:
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [5]:
def summarize_diagnostics(history):
    # plot the loss
    pyplot.subplot(211)
    pyplot.title("Cross Entropy Loss")
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [20]:
def run_test():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    history = model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), verbose=1)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=1)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

    #learning curves
    summarize_diagnostics(history)
    model.save("cifar_model_10.h5")


In [21]:
run_test()

Train : X = (50000, 32, 32, 3), Y = [[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]
Test : X = (10000, 32, 32, 3), Y = [[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
782/782 [==============================] - 7s 8ms/step - loss: 1.7536 - accuracy: 0.3646 - val_loss: 1.4825 - val_accuracy: 0.4586
Epoch 2/20
782/782 [==============================] - 6s 8ms/step - loss: 1.3980 - accuracy: 0.4970 - val_loss: 1.3059 - val_accuracy: 0.5316
Epoch 3/20
782/782 [==============================] - 6s 8ms/step - loss: 1.2146 - accuracy: 0.5672 - val_loss: 1.2107 - val_accuracy: 0.5586
Epoch 4/20
782/782 [==============================] - 6s 8ms/step - loss: 1.0817 - accuracy: 0.6183 - val_loss: 1.1112 - val_accuracy: 0.6054
Epoch 5/20
782/782 [==============================] - 6s 8ms/step - loss: 0.9790 - accuracy: 0.6560 - val_loss: 0.9962 - val_accuracy: 0.6474
Epoch 6/20
782/782 [==============================] - 6s 8ms/step - loss: 0.8912 - accuracy: 0.6880 - val_loss: 0.9493 - val_accuracy: 0.6680
Epoch 7/20
782/782 [==============================] - 6s 8ms/step - loss: 0.8203 - accuracy: 0.7145 - val_loss: 0.9434 - val_accuracy: 0.6719
Epoch 

In [22]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [37]:
def load_image(filename):

  img = load_img(filename, target_size =(32,32))
  img = img_to_array(img)
  img = img.reshape(1,32,32,3)
  img = img.astype("float32")
  img = img / 255.0
  return img

In [45]:
def run_example():

  img = load_image("/content/deer.jpg")
  model = load_model("/content/cifar_model_10.h5")
  result = model.predict_classes(img)
  cifar_list =['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog',
               'frog', 'horse', 'ship', 'truck']
  index = result[0]
  print(cifar_list[index])



In [46]:
run_example()

deer


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
